## Import Libraries

In [6]:
from pymongo import MongoClient
import json

In [508]:
import pprint

In [7]:
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DBS_NAME = 'mydb'
COLLECTION_NAME = 'projects'
FIELDS = "FeatureCollection"

In [226]:
def dashboard_projects():
    connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
    collection = connection[DBS_NAME][COLLECTION_NAME]
    projects = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

    return projects

In [256]:
dashboard_projects()

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.965463853978, -37.8077644149636]},
   'properties': {'FIELD1': 0,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Old Melbourne Gaol Crime & Justice Experience'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.953478052064, -37.8242574266145]},
   'properties': {'FIELD1': 1,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Polly Woodside'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.967252924593, -37.8266045432686]},
   'properties': {'FIELD1': 2,
    'Theme': 'Place Of Assembly',
    'Sub Theme': 'Art Gallery/Museum',
    'Feature Name': 'Australian Centre for Contemporary Art'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [144.968837288153, -37.8219947335768]},
   'properties'

In [295]:
# Call all documents
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
projects2 = collection.find_one({"type": "FeatureCollection"}, {"_id":0})

In [321]:
client = MongoClient()
db = client.mydb
collection = db.projects

In [319]:
from bson.son import SON
from pymongo import MongoClient

db = MongoClient()
latitude = -37.44609999
longitude = 144.96923385414
max_distance = 1000 #meters    

In [903]:
location = [144.965234, -37.8076]

In [755]:
db.collection.drop_indexes()

In [756]:
for idx in db.collection.list_indexes():
    print (idx)

SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_'), ('ns', 'mydb.collection')])


In [757]:
db.collection.create_index([( "features.geometry.coordinates", "2dsphere" )])

'features.geometry.coordinates_2dsphere'

In [785]:
## Unwind search

pipeline = [{"$unwind": "$features"},
            {"$match":{'features.geometry.coordinates':[144.965463853978, -37.8077644149636]}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline)))

[]


In [787]:
pipeline2 = [{"$unwind": "$features"},
            {"$match":{'features.properties.Sub Theme' : 'Art Gallery/Museum'}},
            {"$group": {"_id": "$features"}}]

pprint.pprint(list(collection.aggregate(pipeline2)))

[{'_id': {'geometry': {'coordinates': [144.969070431323, -37.8176107327202],
                       'type': 'Point'},
          'properties': {'FIELD1': 17,
                         'Feature Name': 'Australian Centre For The Moving '
                                         'Image (ACMI)',
                         'Sub Theme': 'Art Gallery/Museum',
                         'Theme': 'Place Of Assembly'},
          'type': 'Feature'}},
 {'_id': {'geometry': {'coordinates': [144.971521914952, -37.80460262432],
                       'type': 'Point'},
          'properties': {'FIELD1': 16,
                         'Feature Name': 'Royal Exhibition Building',
                         'Sub Theme': 'Art Gallery/Museum',
                         'Theme': 'Place Of Assembly'},
          'type': 'Feature'}},
 {'_id': {'geometry': {'coordinates': [144.979471339826, -37.8144604899837],
                       'type': 'Point'},
          'properties': {'FIELD1': 15,
                         'Feature

## Working code

In [904]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : location },
             'distanceField': "dist.calculated",
             'maxDistance': 100,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5ca63a65122b9a6192720138'),
  'dist': {'calculated': 27.27001406965783,
           'location': [144.965463853978, -37.8077644149636]},
  'features': {'geometry': {'coordinates': [144.965463853978,
                                            -37.8077644149636],
                            'type': 'Point'},
               'properties': {'FIELD1': 0,
                              'Feature Name': 'Old Melbourne Gaol Crime & '
                                              'Justice Experience',
                              'Sub Theme': 'Art Gallery/Museum',
                              'Theme': 'Place Of Assembly'},
               'type': 'Feature'},
  'type': 'FeatureCollection'}]


### Check

In [820]:
a1 = [144.9654, -37.8076]
a2 = [144.9656988, -37.80619915]

distance_finder(a1, a2)

157.69509130741474 m


## Implement Automation

In [929]:
click = [144.9607616045045, -37.811441609877804]

In [920]:
location = [144.965234, -37.8076]

In [931]:
pipeline = [
        {'$geoNear': {
            'near' : { 'type' : "Point", 'coordinates' : click },
             'distanceField': "dist.calculated",
             'maxDistance': 500,
             'includeLocs': "dist.location",
              'key': 'features.geometry.coordinates',
             'uniqueDocs': False,
             'spherical': True}},
    { "$unwind" : "$features" },
    { "$redact" : {   
        "$cond" : {
            "if" : { "$eq" : [ { "$cmp" : [ "$features.geometry.coordinates", "$dist.location" ] }, 0 ] },
            "then" : "$$KEEP",
            "else" : "$$PRUNE"
        }
    }
}
]

pprint.pprint(list(collection.aggregate(pipeline)))

[{'_id': ObjectId('5cb0a55fe39a5cbeb4a206d7'),
  'dist': {'calculated': 68.10091452076718,
           'location': [144.9602499, -37.81190077]},
  'features': {'geometry': {'coordinates': [144.9602499, -37.81190077],
                            'type': 'Point'},
               'properties': {'Base property ID': 106018,
                              'Block ID': 74,
                              'CLUE small area': 'Melbourne (CBD)',
                              'Census year': 2015,
                              'FIELD1': 67,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 50,
                              'Property ID': 106018,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '395-397 Little Lonsdale '
      

In [932]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]
pprint.pprint(list(collection.aggregate(pipeline)))
connection.close()

[{'_id': ObjectId('5cb0a55fe39a5cbeb4a206d7'),
  'dist': {'calculated': 68.10091452076718,
           'location': [144.9602499, -37.81190077]},
  'features': {'geometry': {'coordinates': [144.9602499, -37.81190077],
                            'type': 'Point'},
               'properties': {'Base property ID': 106018,
                              'Block ID': 74,
                              'CLUE small area': 'Melbourne (CBD)',
                              'Census year': 2015,
                              'FIELD1': 67,
                              'Industry (ANZSIC4) code': 4520,
                              'Industry (ANZSIC4) description': 'Pubs, Taverns '
                                                                'and Bars',
                              'Number of seats': 50,
                              'Property ID': 106018,
                              'Seating type': 'Seats - Indoor',
                              'Street address': '395-397 Little Lonsdale '
      

## Try +-500

In [1]:
click = [-215.04030733337393, -37.8145422437346]

In [55]:
target = [144.96923385414, -37.8107686127655]

In [90]:
projects.get('features')[0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [144.965463853978, -37.8077644149636]},
 'properties': {'FIELD1': 0,
  'Theme': 'Place Of Assembly',
  'Sub Theme': 'Art Gallery/Museum',
  'Feature Name': 'Old Melbourne Gaol Crime & Justice Experience'}}

In [84]:
from geopy import distance

In [808]:
def distance_finder(click, target):
    # find distance in km using 2 lists
    return print(distance.distance((click[1], click[0]), (target[1], target[0])).m, "m")

In [809]:
distance_finder(click, target)

938.7704897560235 m
